In [ ]:
class OOPImageFile(ImageFile.ImageFile):
    format = "OOP"

    def _open(self):
        self.fp: io.FileIO
        reader = FileStream(self.fp)
        oop_header = reader.read(128)
        file_size = reader.read_int(4)

        unk1 = reader.read_byte_array(4)
        # print(unk1)

        count = reader.read_int(4)

        offsets = []

        for _ in range(count):
            offset = reader.read_int(4)
            offsets.append(offset)
            print(f"offset = {offset:X}")

        file_size_dupe = reader.read_int(4)

        reader.seek(offsets[0])

        wpg_header = reader.read_byte_array(16)
        wpg_temp = reader.read_string()

        wpg_magic1 = reader.read_byte_array(20)

        wpg_id = reader.read_string()

        wpg_magic2 = reader.read_byte_array(4)

        wpg_n_images = reader.read_int(4)

        wpg_magicff = reader.read_byte_array(65)
        wpg_unk1 = reader.read_int()

        self._size = (512, 256)


Image.register_open(OOPImageFile.format, OOPImageFile, lambda prefix: prefix[:3] == b"OOP")

Image.register_extensions(
    OOPImageFile.format,
    [".oop", ".1E3EE6FB"]
)

# start = 0
# pbar = tqdm(range(1000000))
# for i in pbar:
#     reader.seek(start)
#     reader.read(i)
#     rem = reader.read_remaining_bytes()
#     if len(rem) <= 0:
#         break
#     pbar.set_postfix_str(len(rem))
#     im_data = io.BytesIO()
#     try:
#         im = Image.open(im_data, formats=None)
#         print(im.width, im.height)
#     except Image.UnidentifiedImageError:
#         continue


# shape = (256, 128)
# # bytes_per_pixel = 1
# # next_read = shape[0]*shape[1]*bytes_per_pixel
# # print(f'tell={reader.tell()}, total={reader.total_bytes()}, want to read {next_read}')
# # rem_data = reader.read(next_read)
# rem_data = reader.read_remaining_bytes()
# # for mode in Image.MODES:
# #     try:
# #         im = Image.frombytes(mode, (128, 128), rem_data)
# #         plt.figure(figsize=(10, 10))
# #         plt.suptitle(mode)
# #         plt.imshow(im)
# #         plt.tight_layout()
# #     except Exception as ex:
# #         print(f'{mode} runs with exception', ex)

# # im = Image.frombytes("P", (512, 256), rem_data, "raw", "P", 0, -1)
# im = Image.frombytes("RGBA", shape, rem_data)

# # rem_data = reader.read(next_read)
# # im2 = Image.frombytes("L", shape, rem_data)

# arr = np.array(im)
# print(arr.min(), arr.max(), arr.mean(), arr.dtype, arr.shape)
# # im = Image.frombytes("F", (256, 256), rem_data, "bit", 24, 0, 0, 0, -1)
# plt.figure(figsize=(10, 10))
# plt.imshow(im, cmap='gray')
# plt.tight_layout()

# # plt.figure(figsize=(10, 10))
# # plt.imshow(im2, cmap='gray')

# print(f'count={count}, n_images={wpg_n_images}, tell=0x{reader.tell():X}, total={reader.total_bytes()}, rem={len(rem_data)}')

In [240]:
# Extract all OPK files
storage_path = pathlib.Path('textures')

opk_path = pathlib.Path(r'C:\Program Files (x86)\Steam\steamapps\common\Mega Man X Legacy Collection 2\nativeDX10\X8\romPC\data\opk')
for fpath in tqdm(list(opk_path.glob('**/*arc'))):
    output_fpath = opk_path/fpath.parent.stem/fpath.stem/'X8'/'data'/'opk'/fpath.parent.stem/f'{fpath.stem}.1E3EE6FB'
    # subprocess.call(['ARCtool.exe', '-x', '-pc', '-silent', str(output_fpath)], creationflags=subprocess.CREATE_NO_WINDOW)
    assert output_fpath.exists()

    output_fpath.rename(storage_path / output_fpath.name)

100%|██████████| 178/178 [00:00<00:00, 1508.13it/s]


In [333]:
storage_path = pathlib.Path('textures')
for fpath in tqdm(list(storage_path.glob('*.1E3EE6FB'))):
    try:
        with open(fpath, 'rb') as file:
            reader = FileStream(file)

            _ = reader.read_string(8)
            _ = reader.read_int(4)
            _ = reader.read_byte_array(4)
            count = reader.read_int(4)

            offsets = []
            for _ in range(count):
                offset = reader.read_int(4)
                offsets.append(offset)

            _ = reader.read_int(4)
            reader.seek(offsets[0])

            _ = reader.read_byte_array(16)
            _ = reader.read_string()
            _ = reader.read_byte_array(20)
            _ = reader.read_string()
            _ = reader.read_byte_array(4)
            wpg_n_images = reader.read_int()

            if count == 1 and wpg_n_images == 1:
                print('1/1', fpath)
            elif count == 1 and wpg_n_images == 2:
                print('1/2', fpath)
            elif count < 2 and wpg_n_images < 10:
                print(f'{count}/{wpg_n_images} {fpath}')
    except PermissionError:
        continue

100%|██████████| 182/182 [00:00<00:00, 10703.06it/s]

1/1 textures\2D_LOAD_ELEVATOR.1E3EE6FB
1/1 textures\2D_LOAD_ELEVATOR_copy.1E3EE6FB
1/1 textures\2D_LOAD_ONPARE.1E3EE6FB
1/1 textures\2D_LOAD_OP.1E3EE6FB
1/1 textures\2D_LOAD_SIGMA.1E3EE6FB
1/3 textures\coming.1E3EE6FB
1/1 textures\debug.1E3EE6FB
1/7 textures\font - Copy.1E3EE6FB
1/7 textures\font.1E3EE6FB
1/1 textures\HB_001.1E3EE6FB
1/1 textures\HB_002.1E3EE6FB
1/1 textures\HB_002_copy.1E3EE6FB
1/3 textures\HB_006.1E3EE6FB
1/1 textures\HB_013.1E3EE6FB
1/2 textures\intro.1E3EE6FB
1/1 textures\load.1E3EE6FB
1/5 textures\title.1E3EE6FB
1/5 textures\wep_get.1E3EE6FB


In [1]:
import io
import pathlib
import subprocess
import joblib

import matplotlib.pyplot as plt
import multiprocess
import numpy as np
from core.io_util import FileStream
from core.wpg import WPGFile
from PIL import Image, ImageFile
from tqdm import tqdm

def int_list_to_hex(l):
    return [f'{h:X}' for h in l]

SHOW_FIGURES = False
DEBUG = False

figures_dir = pathlib.Path('figures2')
if not figures_dir.exists():
    figures_dir.mkdir()

all_remains = []
failed_files = []
ran_out_of_bytes = []

D1 = 1024

log_path = pathlib.Path('figures2/log.txt')
log_file = open(log_path, 'w')
def printd(st):
    log_file.write(st + '\n')

if DEBUG:
    l = []
    # l += [pathlib.Path('/home/jperez/data/textures/2D_LOAD_ATARI00.1E3EE6FB')]

    l += [pathlib.Path('/home/jperez/data/textures/cockpit.1E3EE6FB')]
    # if not DEBUG_ONE:
    # l += [pathlib.Path('/home/jperez/data/textures/2D_LOAD_ELEVATOR - Copy.1E3EE6FB')]
    # l += [pathlib.Path('/home/jperez/data/textures/2D_LOAD_SIGMA.1E3EE6FB')]
else:
    l = tqdm(list(pathlib.Path('/home/jperez/data/textures').glob('*.1E3EE6FB')))

for op_file in l:
    try:
        file = open(op_file, 'rb')
    except PermissionError:
        file.close()
        printd('Cannot open due to permission')
        continue
    
    try:
        reader = FileStream(file)

        oop_header = reader.read_string(8)
        file_size = reader.read_int(4)
        unk1 = reader.read_byte_array(4)

        count = reader.read_int(4)
        printd(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        printd(f'>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        printd(f'{count} files in {op_file.name} | unk1={unk1}')

        offsets = []
        for _ in range(count):
            offset = reader.read_int(4)
            offsets.append(offset)

        file_size_dupe = reader.read_int(4)

        all_n_images = {}
        for file_idx, offset in enumerate(offsets):
            reader.seek(offset)

            all_extra_ints = {}
            all_shapes = {}
            all_bpps = {}
            all_alt_w = {}
            all_alt_h = {}
            all_wpg_unk3 = {}
            
            wpg_header = reader.read_byte_array(16)
            assert wpg_header[-4:] == [108, 0, 1, 1]

            wpg_temp = reader.read_string()

            wpg_magic1 = reader.read_byte_array(20)

            wpg_id = reader.read_string()

            wpg_magic2 = reader.read_byte_array(4)

            wpg_n_images = reader.read_int(4)
            all_n_images[offset] = wpg_n_images
            printd(f'[File{file_idx+1}/{len(offsets)}] at {offset} with {wpg_n_images} images')
            printd(f'Header={wpg_header}')
            printd(f'Temp={wpg_temp}, ID={wpg_id}, Magic2={wpg_magic2}')

            wpg_unk1_p1 = reader.read_int_array(64, 1)
            assert wpg_unk1_p1 == [255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            # printd(f'\tunk1_p1={wpg_unk1_p1}')

            wpg_unk1_p2 = reader.read_int_array(20, 1)
            reader.seek(reader.tell()-20)
            wpg_unk1_p2_b2 = reader.read_int_array(10)

            printd(f'\tunk1_p2={wpg_unk1_p2}')
            printd(f'\tunk1_p2(hex)={int_list_to_hex(wpg_unk1_p2)}')
            printd(f'\tunk1_p2_b2{wpg_unk1_p2_b2}')
            printd(f'\tunk1_p2_b2(hex)={int_list_to_hex(wpg_unk1_p2_b2)}')

            filler = 0
            for i in range(wpg_n_images):
                printd(f'\t[im{i+1}/{wpg_n_images}]')
                width = reader.read_int()
                height = reader.read_int()
                printd(f'\t[shape] {width}x{height}')

                deb = reader.read_int_array(4, 1)
                reader.seek(reader.tell()-4)
                deb_b2 = reader.read_int_array(2)
                reader.seek(reader.tell()-4)
                printd(f'\t[bpp_debug] {deb}')
                printd(f'\t[bpp_debug_b2] {deb_b2}')

                alt_w = reader.read_int()
                if alt_w == 0 or alt_w % 2 != 0:
                    reader.seek(reader.tell()-2)
                    alt_w2 = reader.read_int(1)
                    reader.read_byte()
                    printd(f'\t\t[alt] Changing w={alt_w} to {alt_w2}')
                    alt_w = alt_w2

                alt_h = reader.read_int()
                if alt_h == 0 or alt_h % 2 != 0:
                    reader.seek(reader.tell()-2)
                    alt_h2 = reader.read_int(1)
                    reader.read_byte()
                    printd(f'\t\t[alt] Changing h={alt_h} to {alt_h2}')
                    alt_h = alt_h2

                wpg_unk2 = reader.read_int_array(4)
                assert wpg_unk2 == [0, 0, 2, 0]
                # printd(f'\t\tunk2={wpg_unk2}')

                bpp = reader.read_int()
                bpp_dupe = reader.read_int()

                if bpp == 8:
                    s1 = min(alt_w, width) if alt_w != 0 else width
                    s2 = min(alt_h, height) if alt_h != 0 else height

                    printd(f'\t\t[bpp8] Changing w={width} to {s1} and h={height} to {s2}')
                    width = s1
                    height = s2

                wpg_unk3 = reader.read_int_array(12, 1)
                reader.seek(reader.tell()-12)
                wpg_unk3_b2 = reader.read_int_array(6)

                printd(f'\t\tunk3={int_list_to_hex(wpg_unk3)}')
                printd(f'\t\tunk3_b2{int_list_to_hex(wpg_unk3_b2)}')
                # printd(f'\tw={width}, h={height}, altw={alt_w}, alth={alt_h}, unk2={wpg_unk2}, bpp={bpp}, bpp_dupe={bpp_dupe}, wpg_unk3={wpg_unk3}, tell={reader.tell()}')
                assert bpp == bpp_dupe

                all_shapes[i] = (width, height)
                all_bpps[i] = bpp
                all_alt_w[i] = alt_w
                all_alt_h[i] = alt_h
                all_wpg_unk3[i] = wpg_unk3

                D1 = 1024 if bpp==16 else 256
                filler += (width * height)+D1

            # Test1
            if file_idx==0:
                initial = reader.read(44)
                reader.seek(reader.tell()-44)
                joblib.dump(initial, 'figures2/initial.data')

            # Test2
            wpg_offd = wpg_unk1_p2[1]
            printd(f'\tWant to add {wpg_offd} to offset')
            next_off = reader.total_bytes() if file_idx == len(offsets)-1 else offsets[file_idx+1]
            dn = next_off-(reader.tell()+filler)+wpg_offd
            printd(f'\tHow about {dn}')

            if file_idx != len(offsets)-1:
                reader.seek(reader.tell()+dn)
            else:
                reader.seek(reader.tell()+dn-wpg_offd)

            predicted_rem_bytes = next_off-(reader.tell()+filler)
            printd(f'\tWe will be {predicted_rem_bytes} bytes away at the end')

            next_bpp = np.unique([val for _, val in all_bpps.items()])
            assert len(next_bpp)==1
            next_bpp = next_bpp[0]

            for i in range(wpg_n_images):
                bpp_mode = all_bpps[i]
                shape = all_shapes[i]

                want = (shape[0]*shape[1])+D1
                printd(f'\t\tim={i}, w={shape[0]}, h={shape[1]}, tell={reader.tell()}/{reader.total_bytes()}, want={want}, bpp={bpp_mode}')

                checkpt = reader.tell()
                im_data = reader.read(want)
                printd(f'\t\t\tData after {reader.tell()}')

                reader.seek(reader.total_bytes()-1024)
                dt2 = reader.read(1024)

                joblib.dump(im_data, f'{figures_dir}/{op_file.stem}_file{file_idx}_im{i}_bpp{bpp}.data')
                # joblib.dump(dt2, f'{figures_dir}/d2.data')
                reader.seek(checkpt+want)
                
                if not im_data:
                    ran_out_of_bytes.append(op_file)
                    printd('RAN OUT OF BYTES')
                    break
                
                after = reader.tell()
                im = Image.frombytes("P", shape, im_data, 'raw', 'P', 0, -1)

                if bpp == 16 or bpp == 8:
                    pal_len = 1024

                    pal_offset = wpg_offd if predicted_rem_bytes < 0 else 0                  
                    reader.seek(after-pal_len-pal_offset)
                    arr = reader.read_int_array(pal_len, size_bytes=1)
                    reader.seek(after)
                    im.putpalette(arr, 'RGBA')
                im = im.convert('RGB')
                im.save(f'{figures_dir}/{op_file.stem}_file{file_idx}_im{i}_bpp{bpp}.png')
            
                if SHOW_FIGURES:
                    fig, curr = plt.subplots(figsize=(35, 35))
                    curr.set_xticklabels([])
                    curr.set_yticklabels([])
                    curr.imshow(im)
            
            rem = next_off-reader.tell()
            all_remains.append(rem)
            printd(f'\tFinished offset at {reader.tell()}/{reader.total_bytes()}. Next offset {next_off} is {rem} bytes away')

        file.close()
    except Exception as ex:
        failed_files.append(op_file)
        if DEBUG:
            raise ex
        continue

print('Remains', np.unique(all_remains))
printd(f'Remains {np.unique(all_remains)}')
if len(failed_files) > 0:
    print('Failed', len(failed_files), failed_files)
if len(ran_out_of_bytes) > 0:
    print('Ran out of bytes', len(ran_out_of_bytes), ran_out_of_bytes)

log_file.close()
# Remains [     0     76    140    172    204 165684]
# Failed 142
# Ran out of bytes 33

100%|██████████| 182/182 [00:00<00:00, 21883.53it/s]

Remains []
Failed 182 [PosixPath('/home/jperez/data/textures/2D_LOAD_ATARI02.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST00_01.1E3EE6FB'), PosixPath('/home/jperez/data/textures/IN_05.1E3EE6FB'), PosixPath('/home/jperez/data/textures/HB_002.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST06_01.1E3EE6FB'), PosixPath('/home/jperez/data/textures/el09_02.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST02_02.1E3EE6FB'), PosixPath('/home/jperez/data/textures/sl09_02.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST05_01.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST10_01.1E3EE6FB'), PosixPath('/home/jperez/data/textures/SL00_02.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST05_03.1E3EE6FB'), PosixPath('/home/jperez/data/textures/ST09_03.1E3EE6FB'), PosixPath('/home/jperez/data/textures/SC_10B.1E3EE6FB'), PosixPath('/home/jperez/data/textures/SL06_01.1E3EE6FB'), PosixPath('/home/jperez/data/textures/EL08_01.1E3EE6FB'), PosixPath('/home/jperez/data/textures/EM08_02